# Data Exploration

###  OpenStreetMap

In [ ]:
import overpy

api = overpy.Overpass()

In [ ]:
query = """
[out:json];
(
  node["amenity"="restaurant"](52.229,20.944,52.410,21.222);
  node["amenity"="bank"](52.229,20.944,52.410,21.222);
);
out body;
"""

result = api.query(query)

In [6]:
len(result.nodes)

1167

In [23]:
for node in result.nodes[:5]:
    print(f"{node.tags.get('amenity', 'unknown')} at ({node.lat}, {node.lon})")

restaurant at (52.2399471, 21.0619767)
restaurant at (52.2312362, 21.0121087)
restaurant at (52.2340883, 21.0233987)
restaurant at (52.2484456, 21.0142668)
restaurant at (52.2371565, 21.1191810)


### GBIF

In [8]:
from pygbif import occurrences

In [15]:
results = occurrences.search(taxon_key=6, geometry="((52.229,20.944,52.410,21.222))")

HTTPError: 400 Client Error: Bad Request for url: https://api.gbif.org/v1/occurrence/search?geometry=%28%2852.229%2C20.944%2C52.410%2C21.222%29%29&limit=300&offset=0&taxon.key=6

In [ ]:
results

{'offset': 0,
 'limit': 300,
 'endOfRecords': True,
 'count': 0,
 'results': [],
 'facets': []}

In [ ]:
# Wydrukuj kilka wyników
for result in results['results']:
    print(f"Species: {result['species']}, Location: {result['decimalLatitude']}, {result['decimalLongitude']}")

## Dataset Class

In [ ]:
from abc import ABC, abstractmethod
import pandas as pd
import overpy

class ColocationDataset(ABC):
    def __init__(self):
        """
        Base class for colocation datasets.
        """
        self._data = None

    @abstractmethod
    def load_data(self) -> pd.DataFrame:
        """Loads the data from the source."""
        pass

    @property
    def data(self) -> pd.DataFrame:
        """Returns the loaded data."""
        return self._data


class OSMColocationDataset(ColocationDataset):
    def __init__(self, area: tuple, poi_types: list):
        """
        Colocation dataset for OpenStreetMap (OSM) data.

        :param area: Area in the format (min_lat, min_lon, max_lat, max_lon)
        :param poi_types: List of POI types to search for (e.g., ['restaurant', 'bank'])
        """
        super().__init__()
        self._area = area
        self._poi_types = poi_types

    def load_data(self) -> pd.DataFrame:
        """Loads data from OSM using the Overpass API."""
        api = overpy.Overpass()
        
        query = f"""
        [out:json];
        (
            {' '.join([f'node["amenity"="{poi}"]({self._area[0]},{self._area[1]},{self._area[2]},{self._area[3]});' for poi in self._poi_types])}
        );
        out body;
        """
        
        result = api.query(query)
        
        data = []
        for node in result.nodes:
            data.append({
                "id": node.id,
                "type": node.tags.get('amenity', 'unknown'),
                "x": node.lat,
                "y": node.lon
            })
        
        self._data = pd.DataFrame(data)
        return self.data
    
    @property
    def data(self) -> pd.DataFrame:
        """Returns the loaded data."""
        if self._data is None:
            self.load_data()
        return self._data


In [28]:
area = (52.229, 20.944, 52.410, 21.222)  # Warsaw area
poi_types = ["bar", "cafe", "fast_food", "food_court", "ice_cream", "pub", 'restaurant', "college", "library", "research_institute", "school", "university", "parking", "atm", 'bank', "clinic", "doctors", "pharmacy", "veterinary", "casino", "cinema", "events_venue", "nightclub", "theatre", "	police"]

dataset = OSMColocationDataset(area, poi_types)
data = dataset.load_data()

In [36]:
data.shape

(4294, 3)

In [30]:
data.head()

,type,x,y
0,restaurant,52.2399471,21.0619767
1,fast_food,52.2454787,21.0858030
2,fast_food,52.2454434,21.0860156
3,fast_food,52.2451877,21.0825923
4,restaurant,52.2312362,21.0121087


In [ ]:
class ColocationPattern:
    def __init__(self, types: list, participation_index: float, instances: list):
        self._types = tuple(sorted(types))
        self._pi = participation_index
        self._instances = instances

    def __str__(self) -> str:
        return f"Pattern {self._types} (PI={self._pi:.2f}, Instances={len(self._instances)})"

    def to_dict(self) -> dict:
        return {
            "types": self._types,
            "participation_index": self._pi,
            "num_instances": len(self._instances)
        }

In [ ]:
from scipy.spatial import KDTree
from collections import defaultdict
from itertools import combinations
import pandas as pd


class ColocationMiner:
    def __init__(self, radius: float = 0.005, min_prevalence: float = 0.3):
        """
        Colocation miner for discovering colocation patterns.

        :param radius: Radius for neighbor search (in degrees).
        :param min_prevalence: Minimum participation index value for a pattern to be considered significant.
        """
        self.radius = radius
        self.min_prevalence = min_prevalence
        self.patterns = []

    def fit(self, df: pd.DataFrame) -> None:
        self.df = df.reset_index(drop=True)
        self.df['id'] = self.df.index
        self.instances_by_type = {
            t: self.df[self.df['type'] == t] for t in self.df['type'].unique()
        }

        self.neighbor_pairs = self._find_neighbor_pairs()
        self.patterns = self._discover_frequent_patterns()

    def _find_neighbor_pairs(self) -> dict:
        neighbor_dict = defaultdict(list)
        for t1, t2 in combinations(self.instances_by_type.keys(), 2):
            pts1 = self.instances_by_type[t1][['x', 'y']].values
            pts2 = self.instances_by_type[t2][['x', 'y']].values
            ids1 = self.instances_by_type[t1]['id'].values
            ids2 = self.instances_by_type[t2]['id'].values

            tree = KDTree(pts2)
            for i, pt in enumerate(pts1):
                idxs = tree.query_ball_point(pt, self.radius)
                for j in idxs:
                    neighbor_dict[(t1, t2)].append((ids1[i], ids2[j]))

        return neighbor_dict

    def _discover_frequent_patterns(self) -> list[ColocationPattern]:
        patterns = []

        for (t1, t2), pairs in self.neighbor_pairs.items():
            if not pairs:
                continue

            ids1 = set(a for a, _ in pairs)
            ids2 = set(b for _, b in pairs)

            pi1 = len(ids1) / len(self.instances_by_type[t1])
            pi2 = len(ids2) / len(self.instances_by_type[t2])
            pi = min(pi1, pi2)

            if pi >= self.min_prevalence:
                patterns.append(ColocationPattern((t1, t2), pi, pairs))

        return patterns

    def get_patterns(self) -> list[ColocationPattern]:
        return sorted(self.patterns, key=lambda p: -p.pi)

In [37]:
miner = ColocationMiner(radius=0.002, min_prevalence=0.3)
miner.fit(data)
patterns = miner.get_patterns()

In [38]:
patterns_df = pd.DataFrame([p.to_dict() for p in patterns])

In [39]:
patterns_df.shape

(28, 3)

In [41]:
patterns_df

,types,participation_index,num_instances
0,"(atm, fast_food)",0.722330,1750
1,"(cafe, fast_food)",0.672026,1733
2,"(cafe, restaurant)",0.643221,2479
3,"(fast_food, restaurant)",0.622834,3095
4,"(bar, pub)",0.620879,268
5,"(atm, cafe)",0.613592,1071
6,"(atm, pharmacy)",0.610169,460
7,"(atm, bank)",0.578641,609
8,"(atm, restaurant)",0.573904,1721
9,"(fast_food, pharmacy)",0.542373,556


In [42]:
class ColocationMiner:
    def __init__(self, radius=0.005, min_prevalence=0.3):
        self.radius = radius
        self.min_prevalence = min_prevalence
        self.patterns = []

    def fit(self, df: pd.DataFrame):
        self.df = df.reset_index(drop=True)
        self.df['id'] = self.df.index
        self.instances_by_type = {
            t: self.df[self.df['type'] == t] for t in self.df['type'].unique()
        }

        # Zacznij od par 2-elementowych
        self.neighbor_pairs = self._find_neighbor_pairs()
        self.patterns = self._discover_frequent_patterns()

        # Iteruj dla większych wzorców (3, 4, ...)
        k = 3
        while True:
            candidates = self._generate_candidates(k)
            if not candidates:
                break

            # Sprawdź, które kandydaty są częste
            new_patterns = self._discover_frequent_patterns_for_candidates(candidates)
            if not new_patterns:
                break

            self.patterns.extend(new_patterns)
            k += 1

    def _find_neighbor_pairs(self):
        neighbor_dict = defaultdict(list)
        for t1, t2 in combinations(self.instances_by_type.keys(), 2):
            pts1 = self.instances_by_type[t1][['x', 'y']].values
            pts2 = self.instances_by_type[t2][['x', 'y']].values
            ids1 = self.instances_by_type[t1]['id'].values
            ids2 = self.instances_by_type[t2]['id'].values

            tree = KDTree(pts2)
            for i, pt in enumerate(pts1):
                idxs = tree.query_ball_point(pt, self.radius)
                for j in idxs:
                    neighbor_dict[(t1, t2)].append((ids1[i], ids2[j]))

        return neighbor_dict

    def _discover_frequent_patterns(self):
        patterns = []
        for (t1, t2), pairs in self.neighbor_pairs.items():
            if not pairs:
                continue

            ids1 = set(a for a, _ in pairs)
            ids2 = set(b for _, b in pairs)

            pi1 = len(ids1) / len(self.instances_by_type[t1])
            pi2 = len(ids2) / len(self.instances_by_type[t2])
            pi = min(pi1, pi2)

            if pi >= self.min_prevalence:
                patterns.append(ColocationPattern((t1, t2), pi, pairs))

        return patterns

    def _generate_candidates(self, k):
        previous_patterns = [p for p in self.patterns if len(p.types) == k - 1]
        candidates = []

        # Generowanie nowych kandydatów (połączenie podzbiorów)
        for p1, p2 in combinations(previous_patterns, 2):
            # Sprawdzamy, czy mają wspólny podzbiór (k-2 elementów)
            common_types = set(p1.types) & set(p2.types)
            if len(common_types) == k - 2:
                new_pattern = tuple(sorted(set(p1.types) | set(p2.types)))
                candidates.append(new_pattern)

        return candidates

    def _discover_frequent_patterns_for_candidates(self, candidates):
        new_patterns = []
        for candidate in candidates:
            # Generujemy instancje
            pairs = self._find_candidate_instances(candidate)
            if pairs:
                pi = self._calculate_participation_index(candidate, pairs)
                if pi >= self.min_prevalence:
                    new_patterns.append(ColocationPattern(candidate, pi, pairs))
        return new_patterns

    def _find_candidate_instances(self, candidate):
        candidate_instances = []

        # Sprawdzanie sąsiedztwa
        for comb in combinations(candidate, 2):
            type1, type2 = comb
            points1 = self.instances_by_type[type1][['x', 'y']].values
            points2 = self.instances_by_type[type2][['x', 'y']].values
            ids1 = self.instances_by_type[type1]['id'].values
            ids2 = self.instances_by_type[type2]['id'].values

            tree = KDTree(points2)
            for i, pt in enumerate(points1):
                idxs = tree.query_ball_point(pt, self.radius)
                for j in idxs:
                    candidate_instances.append((ids1[i], ids2[j]))

        return candidate_instances

    def _calculate_participation_index(self, candidate, pairs):
        """Oblicza participation index dla kandydata."""
        ids = {i for pair in pairs for i in pair}
        pi = len(ids) / len(self.df)
        return pi

    def get_patterns(self):
        return sorted(self.patterns, key=lambda p: -p.pi)


In [43]:
# Testowanie z wcześniej załadowanymi danymi POI
miner = ColocationMiner(radius=0.005, min_prevalence=0.3)
miner.fit(data)
patterns = miner.get_patterns()
patterns_df = pd.DataFrame([p.to_dict() for p in patterns])


KeyboardInterrupt: 